In [1]:
# # %load_ext autoreload
# %autoreload 2
import torch
from transformer_vq.nn.attn_vq import VQAttentionQK
from transformer_vq.nn.config_spec import TransformerConfig
from transformer_vq.nn.emb import TransformerEmbedding as Emb
import yaml
from einops import rearrange
with open('conf.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
n_vocab = config['n_vocab']
sequence_length = config['sequence_len']
batch_size = config['global_batch_size']
d_model = config['d_model']
config['d_type'] = torch.float32
config['param_dtype'] = torch.float32
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
data = torch.randint(low=0, high=n_vocab, size=(4, sequence_length))
d_model = config['d_model']
emb = Emb(n_vocab, d_model)
model_config = TransformerConfig(**config)
model = VQAttentionQK(model_config)
data = emb(data)
block_len = config['block_len']
data = rearrange(data, 'b (l s) d -> l b s d', l=block_len)
# data = rearrange(data, 'b s d -> b (s d)')

In [2]:
data.shape

torch.Size([32, 4, 32, 1024])

In [3]:
block_len

32

In [8]:
q, k, v, g = model.compute_k_q_v_g(data)

In [9]:
q.shape

torch.Size([32, 4, 32, 384])

In [11]:
k.shape

torch.Size([32, 4, 3, 32, 128])

In [12]:
v.shape

torch.Size([32, 4, 3, 32, 128])

In [13]:
g.shape

torch.Size([32, 4, 32, 384])

In [5]:
from transformer_vq.nn.norm import LayerNorm

In [6]:
norm = LayerNorm(d_model, 42)

In [7]:
norm

LayerNorm()